## Explore the data and identify if there are any :
- dublicated values
- missing values
- negative values
- wrong data types
- etc.

In [27]:
# Import libraries

import psycopg2
import pandas as pd
import numpy as np

import boto3
from io import StringIO
import warnings
warnings.filterwarnings("ignore")

In [28]:
# Connect to Redshift
# Fill the Values
# NOTE: NEVER share your passwords!

dbname = 
host = 
port = 
user = 
password = 

connect = psycopg2.connect(dbname=dbname, host=host, port=port, user=user, password=password)
cursor = connect.cursor()

In [31]:
# Create a query for extracting table stock_description using SQL from Redshift

query = '''select * 
        from bootcamp1.stock_description'''

stock_description = pd.read_sql(query, connect)
stock_description.head()

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [30]:
# Create a query for extracting table online_transactions

query = '''select * 
        from bootcamp1.online_transactions'''

online_transactions = pd.read_sql(query, connect)
online_transactions.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom
3,536367,22748,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom
4,536367,22623,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom


## Explore the data

In [10]:
# Check the shape of the table

online_transactions.shape

(541910, 7)

In [11]:
# Check the shape of the table

stock_description.shape

(3952, 2)

In [14]:
# Check if there is duplicated data

online_transactions.duplicated().sum()

5270

In [32]:
# preview the duplicated data
#Note: duplicated data schould be removed!

online_transactions[online_transactions.duplicated()]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
296,536412,22327,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
298,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
313,536412,21708,1,2010-12-01 11:49:00,4.95,u1792,United Kingdom
314,536412,85184C,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
372,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom
...,...,...,...,...,...,...,...
541479,581471,21411,2,2011-12-08 19:29:00,1.95,u14702,United Kingdom
541835,581514,22075,24,2011-12-09 11:20:00,0.39,u17754,United Kingdom
541846,581538,23275,1,2011-12-09 11:34:00,1.25,u14446,United Kingdom
541851,581538,22068,1,2011-12-09 11:34:00,0.39,u14446,United Kingdom


In [15]:
# Check if there are null values

online_transactions.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [16]:
# Check object types
# Note: invoice_date data type schould be changed to date format!

online_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [17]:
# Get summary statistics for numerical columns of our DataFrame

online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [33]:
# Check if there is duplicated data

stock_description.duplicated().sum()

0

In [34]:
# Check if there are null values

stock_description.isnull().sum()

stock_code     0
description    0
dtype: int64

In [35]:
# Get summary statistics
#Note: this function is mostly used with numerical data, but in our case helped us identify '?' as a description name, witch should be renamed as 'Unknown'

stock_description.describe()

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [36]:
# Check if there is any other duplicated data
# we have ?, and 6 stock codes with the same description and other duplicates appearing.

stock_description.description.value_counts()

?                                     47
METAL SIGN,CUPCAKE SINGLE HOOK         6
CINAMMON SET OF 9 T-LIGHTS             2
COLUMBIAN CANDLE RECTANGLE             2
3 WHITE CHOC MORRIS BOXED CANDLES      2
                                      ..
MAGNETS PACK OF 4 SWALLOWS             1
MAGNETS PACK OF 4 CHILDHOOD MEMORY     1
MAGNETS PACK OF 4 HOME SWEET HOME      1
MAGNETS PACK OF 4 VINTAGE COLLAGE      1
SAMPLES                                1
Name: description, Length: 3785, dtype: int64

In [37]:
# Check stock codes where description is ?

stock_description[stock_description.description == "?"]

,stock_code,description
36,16020C,?
64,16207B,?
390,21145,?
451,21232,?
535,21368,?
584,21427,?
594,21446,?
675,21591,?
865,21877,?
997,22077,?


In [38]:
# Check invoices where price is less than 0

online_transactions[online_transactions.price < 0]

#NOTE: it appears a missing customer_id

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
270865,A563187,B,1,2011-08-12 14:52:00,-11062.06,,United Kingdom
372989,A563186,B,1,2011-08-12 14:51:00,-11062.06,,United Kingdom


In [39]:
# Check invoices where quantity is less than 0

online_transactions[online_transactions.quantity < 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
58,C536391,22556,-12,2010-12-01 10:24:00,1.65,u17548,United Kingdom
59,C536391,21484,-12,2010-12-01 10:24:00,3.45,u17548,United Kingdom
218,C536391,21984,-24,2010-12-01 10:24:00,0.29,u17548,United Kingdom
219,C536391,22557,-12,2010-12-01 10:24:00,1.65,u17548,United Kingdom
731,C536548,22242,-5,2010-12-01 14:33:00,1.65,u12472,Germany
...,...,...,...,...,...,...,...
541453,C581465,23660,-2,2011-12-08 18:59:00,1.65,u15755,United Kingdom
541454,C581466,22838,-1,2011-12-08 19:20:00,14.95,u13883,United Kingdom
541455,C581466,21232,-1,2011-12-08 19:20:00,1.25,u13883,United Kingdom
541468,C581470,23084,-4,2011-12-08 19:28:00,2.08,u17924,United Kingdom


In [40]:
# Check invoice that has a minimum negative quantity less than 0

online_transactions.quantity.min()

-80995

In [41]:
# Check for missing data again

online_transactions.isnull().sum()

# Looks like when migrating the data to redshift, the missing data was not handled correctly, as we've just identified the missing customer_id

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [42]:
# Fix this by replacing the blanks with python recognised nan

online_transactions = online_transactions.replace('', np.nan) # to get rid of empty values

In [43]:
# Check for missing values again

online_transactions.isnull().sum()

invoice              0
stock_code           0
quantity             0
invoice_date         0
price                0
customer_id     135080
country              0
dtype: int64

In [44]:
# Check missing data in stock description table

stock_description.isnull().sum()

stock_code     0
description    0
dtype: int64

In [45]:
# fix missing data

stock_description = stock_description.replace('', np.nan)   
stock_description.isnull().sum()

stock_code     0
description    0
dtype: int64

In [46]:
# Check the number of stock codes

len(online_transactions.stock_code.unique())

4070

In [47]:
# Check the number of stock codes

len(stock_description.stock_code.unique())

3905

In [48]:
# writing a query to identify how many stock codes have been purchased but are not in the stock description database

query = """select count(distinct t1.stock_code)
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)

,count
0,165


In [51]:
# Check the data where stock codes are missing in the stock description table

query = """select *
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)

# NOTE: there is a missing customer id

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,stock_code,description
0,536544,21704,1,2010-12-01 14:32:00,1.66,,United Kingdom,None,None
1,536549,85226A,1,2010-12-01 14:34:00,0.00,,United Kingdom,None,None
2,536557,46000S,1,2010-12-01 14:41:00,1.45,u17841,United Kingdom,None,None
3,536569,M,1,2010-12-01 15:35:00,1.25,u16274,United Kingdom,None,None
4,536592,21594,1,2010-12-01 17:06:00,4.21,,United Kingdom,None,None
...,...,...,...,...,...,...,...,...,...
2374,581405,M,3,2011-12-08 13:50:00,0.42,u13521,United Kingdom,None,None
2375,581406,46000S,300,2011-12-08 13:58:00,0.00,,United Kingdom,None,None
2376,581475,21705,24,2011-12-09 08:39:00,0.39,u13069,United Kingdom,None,None
2377,581492,21703,1,2011-12-09 10:03:00,0.83,,United Kingdom,None,None


In [52]:
# Check the data where stock codes are missing in the stock description table
# Filter out the missing customer_id

query = """select count(distinct t1.stock_code)
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

,count
0,22


In [53]:
# Have a closer look at the data where stock codes are missing in the stock description table

query = """select distinct t1.stock_code
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

,stock_code
0,M
1,46000U
2,46000R
3,20964
4,21703
5,46000P
6,22686
7,22878
8,D
9,21704


In [54]:
# Close the connection to Redshift

connect.close()

## Results:

- 5270 rows of duplicated data - remove duplicated rows and keep just one instance
- missing customer id
- invoice_date field has a wrong data type
- 22 stock codes without customer_id do not have a descriptions # may need further investigation
- 10k invoices with a negative quantity - may be considered to remove this rows
- invoices with negative price and missing customer id
- 47 examples of the description having a ?, which we will be replace with Unknown